# Parcel Distribution Center Example - Perpetual Simulation

This example demonstrates a long-running Petri net process where parcels continuously arrive, get sorted, staged, and dispatched. The process runs indefinitely until interrupted.

## Process Flow

1. **Receive Parcel** → Parcels arrive from delivery trucks (every 1 second)
2. **Receiving** → Parcels are sorted
3. **Sorting** → Parcels move to staging area
4. **Staging Area** → Delivery trucks depart (every 5 seconds, randomly some parcels get left behind for the next truck)

This demonstrates:
- **Perpetual execution**: Parcels continuously arrive with no finite input
- **Time-based activation**: Transitions controlled by activation functions (1s for arrivals, 5s for dispatch)
- **Non-blocking timing**: Other transitions can fire while waiting for timed transitions
- **Guard-based selection**: Custom selector that respects activation function guards
- **Probabilistic behavior**: Random parcel left-behinds (1 in 10 chance each parcel gets left for next truck)
- **Long-running processes**: Ideal for testing server-based execution where the process runs independently

## Imports and Setup

In [ ]:
import random
import time
from typing import Optional
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from pydantic import BaseModel

from petritype.core.executable_graph_components import (
    ExecutableGraph,
    ListPlaceNode,
    FunctionTransitionNode,
    ArgumentEdgeToTransition,
    ReturnedEdgeFromTransition,
    ExecutableGraphOperations,
)
from petritype.core.rustworkx_graph import RustworkxGraph
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization

## Define Domain Model

We define a simple type:
- `Parcel`: A parcel with a unique tracking ID

We'll use a global counter to generate unique IDs for each parcel.

In [ ]:
# Global counter for parcel tracking IDs
_parcel_counter = 0

# Timing state for activation functions
_last_arrival_time = 0.0
_last_dispatch_time = 0.0


class Parcel(BaseModel):
    """A parcel with a tracking ID."""
    id: int

## Define Transition Functions and Activation Functions

Each transition represents a step in the distribution process.

Note that `receive_parcel()` has **no parameters** - it generates parcels from nothing (simulating continuous arrivals)!

**Activation Functions**: We use activation functions to control timing:
- `can_receive_parcel()`: Only allows receiving a parcel every 1 second
- `can_dispatch_truck()`: Only allows truck dispatch every 5 seconds

The timing is controlled by activation functions, not by sleeping in the transition functions themselves. This allows other transitions to fire while waiting.

In [ ]:
def receive_parcel() -> Parcel:
    """Receive a parcel from a delivery truck (no delay in function itself).
    
    The timing is controlled by the activation function.
    """
    global _parcel_counter, _last_arrival_time
    _parcel_counter += 1
    _last_arrival_time = time.time()
    print(f"📦 Receiving parcel #{_parcel_counter}...")
    return Parcel(id=_parcel_counter)


def can_receive_parcel() -> bool:
    """Activation function: only allow receiving a parcel every 1 second."""
    global _last_arrival_time
    elapsed = time.time() - _last_arrival_time
    return elapsed >= 1.0


def sort_parcel(parcel: Parcel) -> Parcel:
    """Move a parcel to Sorting."""
    print(f"  Parcel #{parcel.id} → Sorting...")
    return parcel


def stage_parcel(parcel: Parcel) -> Parcel:
    """Move a parcel to Staging Area."""
    print(f"  Parcel #{parcel.id} → Staging Area")
    return parcel


def dispatch_truck(parcels: list[Parcel]) -> list[Parcel]:
    """Dispatch delivery truck - randomly some parcels get left behind.
    
    Each parcel has a 1 in 10 chance of being left for the next truck.
    Returns the parcels that got left behind.
    The timing is controlled by the activation function.
    """
    global _last_dispatch_time
    _last_dispatch_time = time.time()
    
    left_behind = []
    dispatched_ids = []
    
    for parcel in parcels:
        if random.randint(1, 10) == 1:
            # Parcel gets left behind for next truck
            left_behind.append(parcel)
        else:
            # Parcel dispatched successfully
            dispatched_ids.append(parcel.id)
    
    if dispatched_ids:
        print(f"🚚 Truck departed with parcels: {dispatched_ids}")
    if left_behind:
        print(f"⏳ Left behind for next truck: {[p.id for p in left_behind]}")
    
    return left_behind


def can_dispatch_truck() -> bool:
    """Activation function: only allow truck dispatch every 5 seconds."""
    global _last_dispatch_time
    elapsed = time.time() - _last_dispatch_time
    return elapsed >= 5.0

## Create the Petri Net Graph with Custom Selector

This graph has no initial tokens - the "Receive Parcel" transition generates tokens from nothing, creating a perpetual process.

We use a **guard-based selector** that respects activation functions. The selector checks each enabled transition's activation function. If it returns `True`, that transition can fire. If `False`, the transition is blocked even though it has sufficient tokens.

In [ ]:
def guard_based_selector(graph: ExecutableGraph, enabled: list[FunctionTransitionNode]):
    """Selector that respects activation functions as guards.
    
    Tries each enabled transition's activation function. If it returns True,
    that transition is selected. If no activation function, transition is always ready.
    """
    for transition in enabled:
        if transition.activation_function is None:
            # No guard - always ready
            return transition
        
        # Check guard
        if transition.activation_function():
            return transition
    
    # All guards blocked
    return None


# Define the graph structure
executable_graph_nodes_and_edges = [
    # Receive Parcel transition - parcels arrive every 1 second
    # No input edges - this generates tokens continuously!
    FunctionTransitionNode(
        name="Receive Parcel",
        function=receive_parcel,
        activation_function=can_receive_parcel,  # Only fire every 1 second
    ),
    ReturnedEdgeFromTransition("Receive Parcel", "Receiving"),
    
    # Receiving place - holds newly arrived parcels
    ListPlaceNode(name="Receiving", type=Parcel),
    
    # Sort transition - moves parcel to Sorting (instant)
    ArgumentEdgeToTransition("Receiving", "Sort", "parcel"),
    FunctionTransitionNode(
        name="Sort",
        function=sort_parcel,
    ),
    ReturnedEdgeFromTransition("Sort", "Sorting"),
    
    # Sorting place
    ListPlaceNode(name="Sorting", type=Parcel),
    
    # Stage transition - moves parcel to Staging Area (instant)
    ArgumentEdgeToTransition("Sorting", "Stage", "parcel"),
    FunctionTransitionNode(
        name="Stage",
        function=stage_parcel,
    ),
    ReturnedEdgeFromTransition("Stage", "Staging Area"),
    
    # Staging Area place
    ListPlaceNode(name="Staging Area", type=Parcel),
    
    # Dispatch Truck transition - fires every 5 seconds
    ArgumentEdgeToTransition("Staging Area", "Dispatch Truck", "parcels"),
    FunctionTransitionNode(
        name="Dispatch Truck",
        function=dispatch_truck,
        activation_function=can_dispatch_truck,  # Only fire every 5 seconds
    ),
    ReturnedEdgeFromTransition("Dispatch Truck", "Staging Area"),
]

In [ ]:
executable_graph = ExecutableGraphOperations.construct_graph(executable_graph_nodes_and_edges)

# Set the selector that respects activation functions
executable_graph.transition_selector = guard_based_selector

executable_pydigraph = RustworkxGraph.from_executable_graph(executable_graph)
SimpleGraphvizVisualization.graph(executable_pydigraph)

## Key Observations

- **Perpetual execution**: The "Receive Parcel" transition has no inputs, so it can always fire - creating infinite parcels!
- **Token generation from nothing**: Transitions don't always need input tokens - they can create tokens ex nihilo
- **Time-based activation**: Transitions fire at controlled intervals (1s for arrivals, 5s for dispatch) using activation functions
- **Guard-based selection**: The custom selector respects activation functions as guards - transitions are blocked even with sufficient tokens
- **Non-blocking timing**: Timing is controlled by guards, not by sleeping in transitions - other transitions can fire while waiting
- **Probabilistic behavior**: The "Dispatch Truck" transition has a 10% chance of leaving each parcel behind for the next truck
- **Token flow**: Parcels move through multiple states: Receiving → Sorting → Staging Area → (dispatched or left behind)
- **All-tokens pattern**: The "Dispatch Truck" transition takes ALL parcels from staging at once (`list[Parcel]`)

This pattern is ideal for:
- **Long-running server processes**: Can run indefinitely in the background
- **Event generation**: Creating events/tokens continuously at controlled rates
- **Time-based simulations**: Modeling processes with realistic timing without blocking
- **Testing distributed execution**: Perfect for validating that Petri nets can run independently of observers
- **Custom transition selection**: Demonstrates how selectors and activation functions enable sophisticated control flow
- **Distribution center operations**: Models real-world logistics where items arrive continuously and are processed in batches

## Execute the Petri Net with Animation

Watch as:
1. Parcels arrive every 1 second (controlled by activation function)
2. Parcels move through Sorting instantly
3. Parcels move to Staging Area instantly
4. Trucks dispatch every 5 seconds (controlled by activation function), with some parcels randomly left behind

**Time-based control**: The timing is controlled by activation functions, not by sleeping in transitions. This means:
- Other transitions can fire while waiting for timed transitions
- The selector checks activation functions each cycle
- Transitions are blocked by their guards, not by token availability

**Note**: This process runs **indefinitely** - it will never stop on its own! You'll need to interrupt the kernel to stop it.

**Tip**: Limit the execution to a few steps for demonstration purposes.

In [ ]:
# Limit number of steps for demonstration (remove this limit to run indefinitely)
max_steps = 800

async for step, diagram, transitions_fired in SimpleGraphvizVisualization.animate_execution_generator(
    executable_graph=executable_graph,
    executable_pydigraph=executable_pydigraph,
):
    clear_output(wait=True)
    print(f"Step {step}")
    display(diagram)
    print(f"Transitions fired: {transitions_fired}")
    
    if step >= max_steps:
        print(f"\\nReached {max_steps} steps - stopping for demonstration.")
        print("This process would normally run indefinitely!")
        break
    
    time.sleep(0.5)
    plt.close()